# Poravnanja sekvenci

In [2]:
class Alignments:
    def __init__(self, gap_penalty = -1, match_score = 1, missmatch_penalty = 0, sigma = -1, eps = -0.5):
        self.GAP_PENALTY = gap_penalty
        self.MATCH_SCORE = match_score
        self.MISSMATCH_PENALTY = missmatch_penalty
        self.SIGMA = sigma
        self.EPS = eps
    
    '''
    Pomoćni metod za proveru jednakosti karaktera
    koji vraća 1 ako su karakteri jednaki, 0 inače
    '''
    def match(self, x, y):
        return int(x == y)
    
    '''
    Pronalaženje najduže zajedničke podsekvence
    LCS - Longest Common Subsequence
    '''
    def lcs_backtrack(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        s = [[0 for _ in range(m)] for _ in range(n)]
        backtrack = [[None for _ in range(m)] for _ in range(n)]
        
        # Inicijalizacija
        # s[i][0] = 0, s[0][j] = 0
        for i in range(1, n):
            backtrack[i][0] = (i - 1, 0)
            
        for j in range(1, m):
            backtrack[0][j] = (0, j - 1)
        
        for i in range(1, n):
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                
                from_up = s[i - 1][j]
                from_left = s[i][j - 1]
                from_diag = s[i - 1][j - 1] + match
                
                s[i][j] = max(
                    from_up,
                    from_left,
                    from_diag,
                )
                
                if match == 1 and s[i][j] == from_diag:
                    backtrack[i][j] = (i - 1, j - 1)
                elif s[i][j] == from_up:
                    backtrack[i][j] = (i - 1, j)
                else:
                    backtrack[i][j] = (i, j - 1)
                
        lcs = ['' for _ in range(s[n - 1][m - 1])]
        
        i = n - 1
        j = m - 1
        k = s[n - 1][m - 1] - 1
        
        while backtrack[i][j] != None:
            (i_new, j_new) = backtrack[i][j]
            if (i_new, j_new) == (i - 1, j - 1):
                lcs[k] = v[i - 1]
                k -= 1
            i = i_new
            j = j_new
            
        
        return s[n - 1][m - 1], ''.join(lcs)
    
    '''Metod za rekonstrukciju poravnanja'''
    def backtracking(self, backtrack, s, v, w, i, j, stop_condition=None):
        v_align = ''
        w_align = ''
        
        while backtrack[i][j] != None and s[i][j] != stop_condition:
            (i_new, j_new) = backtrack[i][j]
            if (i_new, j_new) == (i - 1, j - 1):
                v_align = v[i - 1] + v_align
                w_align = w[j - 1] + w_align

            elif (i_new, j_new) == (i - 1, j):
                v_align = v[i - 1] + v_align
                w_align = '-' + w_align
                
            else:
                v_align = '-' + v_align
                w_align = w[j - 1] + w_align
                
            i = i_new
            j = j_new
            
        return v_align, w_align
    
    '''Izračunavanje edit rastojanja između sekvenci'''
    def edit_distance(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        s = [[0 for _ in range(m)] for _ in range(n)]
        backtrack = [[None for _ in range(m)] for _ in range(n)]

        # Initialization
        for i in range(1, n):
            s[i][0] = i
            backtrack[i][0] = (i - 1, 0)
            
        for j in range(1, m):
            s[0][j] = j
            backtrack[0][j] = (0, j - 1)
            
        for i in range(1, n):
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                
                from_up = s[i - 1][j] + 1
                from_left = s[i][j - 1] + 1
                from_diag = s[i - 1][j - 1] + (1 - match)
            
                s[i][j] = min(
                    from_up,
                    from_left,
                    from_diag,
                )
                
                if s[i][j] == from_diag:
                    backtrack[i][j] = (i - 1, j - 1)
                elif s[i][j] == from_up:
                    backtrack[i][j] = (i - 1, j)
                else:
                    backtrack[i][j] = (i, j - 1)
               
        
        v_align, w_align = self.backtracking(backtrack, s, v, w, n - 1, m - 1)
                    
        return s[n - 1][m - 1], v_align, w_align
    
    '''Izračunavanje globalnog poravnanja između sekvenci'''
    def needleman_wunsch(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        s = [[0 for _ in range(m)] for _ in range(n)]
        backtrack = [[None for _ in range(m)] for _ in range(n)]

        # Initialization
        for i in range(1, n):
            s[i][0] = i * self.GAP_PENALTY
            backtrack[i][0] = (i - 1, 0)
            
        for j in range(1, m):
            s[0][j] = j * self.GAP_PENALTY
            backtrack[0][j] = (0, j - 1)
            
        for i in range(1, n):
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                match_score = 0
                
                if match == 1:
                    match_score = self.MATCH_SCORE
                else:
                    match_score = self.MISSMATCH_PENALTY
                
                from_up = s[i - 1][j] + self.GAP_PENALTY
                from_left = s[i][j - 1] + self.GAP_PENALTY
                from_diag = s[i - 1][j - 1] + match_score
            
                s[i][j] = max(
                    from_up,
                    from_left,
                    from_diag,
                )
                
                if s[i][j] == from_diag:
                    backtrack[i][j] = (i - 1, j - 1)
                elif s[i][j] == from_up:
                    backtrack[i][j] = (i - 1, j)
                else:
                    backtrack[i][j] = (i, j - 1)
                    
        v_align, w_align = self.backtracking(backtrack, s, v, w, n - 1, m - 1)
                    
        return s[n - 1][m - 1], v_align, w_align
    
    '''
    Izračunavanje skora globalnog poravnanja
    u linearnom vremenu, metod vraća poslednji
    red matrice poravnanja
    '''
    def needleman_wunsch_last_line(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        s = [[0 for _ in range(m)] for _ in range(2)]
        
        # Inicijalizacija
        for j in range(m):
            s[0][j] = j * self.GAP_PENALTY
       
        for i in range(1, n):
            s[1][0] = i * self.GAP_PENALTY
            
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                match_score = 0
                
                if match == 1:
                    match_score = self.MATCH_SCORE
                else:
                    match_score = self.MISSMATCH_PENALTY
                
                from_up = s[0][j] + self.GAP_PENALTY
                from_left = s[1][j - 1] + self.GAP_PENALTY
                from_diag = s[0][j - 1] + match_score
            
                s[1][j] = max(
                    from_up,
                    from_left,
                    from_diag,
                )

            s[0][:] = s[1][:]
                
        return s[1]
    
    '''
    Metod za izračunavanje globalnog poravnanja
    u linearnom vremenu
    '''
    def hirschberg(self, v, w):
        v_align = ''
        w_align = ''
        
        n = len(v)
        m = len(w)
        
        if n == 0:
            v_align = m * '-'
            w_align = w
            
        elif m == 0:
            v_align = v
            w_align = n * '-'
            
        elif n == 1 or m == 1:
            _, v_align, w_align = self.needleman_wunsch(v, w)
            
        else:
            x_mid = n // 2
            
            score_l = self.needleman_wunsch_last_line(v[:x_mid], w)
            score_r = self.needleman_wunsch_last_line(v[x_mid:][::-1], w[::-1])[::-1]
            
            max_score = float('-inf')
            y_mid = None
            
            for i in range(m):
                curr_score = score_l[i] + score_r[i]
                if curr_score > max_score:
                    max_score = curr_score
                    y_mid = i
                    
            v1_align, w1_align = self.hirschberg(v[:x_mid], w[:y_mid])
            v2_align, w2_align = self.hirschberg(v[x_mid:], w[y_mid:])
            
            v_align = v1_align + v2_align
            w_align = w1_align + w2_align
            
        return v_align, w_align
    
    '''Izračunavanje lokalnog poravnanja između sekvenci'''
    def smith_waterman(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        s = [[0 for _ in range(m)] for _ in range(n)]
        backtrack = [[None for _ in range(m)] for _ in range(n)]

        # Inicijalizacija
        for i in range(1, n):
            # s[i][0] = 0
            backtrack[i][0] = (i - 1, 0)
            
        for j in range(1, m):
            # s[0][j] = 0
            backtrack[0][j] = (0, j - 1)
            
        max_score = 0
        max_score_pos = (0,0)
        
        for i in range(1, n):
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                match_score = 0
                
                if match == 1:
                    match_score = self.MATCH_SCORE
                else:
                    match_score = self.MISSMATCH_PENALTY
                
                from_up = s[i - 1][j] + self.GAP_PENALTY
                from_left = s[i][j - 1] + self.GAP_PENALTY
                from_diag = s[i - 1][j - 1] + match_score
            
                s[i][j] = max(
                    from_up,
                    from_left,
                    from_diag,
                    0,
                )
                
                if s[i][j] >= max_score:
                    max_score = s[i][j]
                    max_score_pos = (i, j)
                
                if s[i][j] == from_diag:
                    backtrack[i][j] = (i - 1, j - 1)
                elif s[i][j] == from_up:
                    backtrack[i][j] = (i - 1, j)
                else:
                    backtrack[i][j] = (i, j - 1)
                    
        (i, j) = max_score_pos
        v_align, w_align = self.backtracking(backtrack, s, v, w, i, j, 0)
                    
        return s[n - 1][m - 1], v_align, w_align
    
    '''
    Pronalaženje poravnanja sekvenci
    korišćenjem afine funkcije za penalizaciju
    praznina u poravnanju
    '''
    def affine_gap_penalty_alignment(self, v, w):
        n = len(v) + 1
        m = len(w) + 1
        
        middle = [[0 for _ in range(m)] for _ in range(n)]
        upper = [[0 for _ in range(m)] for _ in range(n)]
        lower = [[0 for _ in range(m)] for _ in range(n)]
        
        backtrack = [[None for _ in range(m)] for _ in range(n)]

        # Inicijalizacija
        for i in range(1, n):
            backtrack[i][0] = (i - 1, 0)
            
        for j in range(1, m):
            backtrack[0][j] = (0, j - 1)
            
        for i in range(1, n):
            for j in range(1, m):
                match = self.match(v[i - 1], w[j - 1])
                match_score = 0
                
                if match == 1:
                    match_score = self.MATCH_SCORE
                else:
                    match_score = self.MISSMATCH_PENALTY
                
                lower[i][j] = max(
                    lower[i-1][j] + self.EPS,
                    middle[i-1][j] + self.SIGMA
                )
                
                upper[i][j] = max(
                    upper[i][j - 1] + self.EPS,
                    middle[i][j - 1] + self.SIGMA
                )
                
                middle[i][j] = max(
                    middle[i - 1][j - 1] + match_score,
                    lower[i][j],
                    upper[i][j]
                )
                
                if middle[i][j] == middle[i - 1][j - 1] + match_score:
                    backtrack[i][j] = (i - 1, j - 1)
                elif middle[i][j] == lower[i][j]:
                    backtrack[i][j] = (i - 1, j)
                else:
                    backtrack[i][j] = (i, j - 1)
                    
        v_align, w_align = self.backtracking(backtrack, middle, v, w, n - 1, m - 1, None)
                    
        return middle[n - 1][m - 1], v_align, w_align

## Primeri

In [3]:
al = Alignments()

In [4]:
v = 'ABCD'
w = 'ABED'
al.lcs_backtrack(v, w)

(3, 'ABD')

In [5]:
v = 'ABCAD'
w = 'ABDE'
al.edit_distance(v, w)

(3, 'ABCAD', 'AB-DE')

In [7]:
al = Alignments(gap_penalty = -2, match_score = 2, missmatch_penalty = -1)
v = 'AGTACGCA'
w = 'TATGC'
al.needleman_wunsch(v, w)

(1, 'AGTACGCA', '--TATGC-')

In [10]:
al = Alignments(gap_penalty = -2, match_score = 2, missmatch_penalty = -1)
v = 'AGTACGCA'
w = 'TATGC'
al.needleman_wunsch_last_line(v, w)

[-16, -12, -8, -7, -3, 1]

In [11]:
al = Alignments(gap_penalty = -2, match_score = 2, missmatch_penalty = -1)
v = 'AGTACGCA'
w = 'TATGC'
al.hirschberg(v, w)

('AGTACGCA', '--TATGC-')

In [12]:
al = Alignments(gap_penalty = -1, match_score = 1, missmatch_penalty = 0)
v = 'OIUGNOISGRVISDIOGHIODSHFGMLSVEODRGMSOVFOGHLSDGMOVIDFHGS'
w = 'DSHGMLSEOD'
al.smith_waterman(v, w)

(0, 'DSHFGMLSVEOD', 'DSH-GMLS-EOD')

In [13]:
al = Alignments(gap_penalty = -2, match_score = 2, missmatch_penalty = -2, sigma=-1, eps=-0.1)
v = 'AGTACGCA'
w = 'TATGC'
al.affine_gap_penalty_alignment(v, w)

(5, 'AGTACGCA', '--TATGC-')